In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
scotty = pd.read_csv("/kaggle/input/scotty/data/data-train.csv")
scotty.head()

In [ ]:
scotty = scotty[['start_time','src_sub_area']]
#scotty.head()

scotty.info()

scotty.start_time = scotty.start_time.astype('datetime64[ns]')
scotty.columns = ['datetime','src_sub_area']
# print(scotty.head())
# print(scotty.tail())

scotty['datetime'] = scotty['datetime'].apply(lambda x: x.replace(minute=0, second=0))
# print(scotty.head())
# print(scotty.tail())

scotty_group = scotty.groupby(['datetime','src_sub_area']).count()
scotty_group.reset_index(inplace=True)

a = scotty.groupby(['datetime','src_sub_area'])['src_sub_area'].count()
scotty_group['demand'] = a.reset_index(drop=True)
# scotty_group.head()

scotty_group.sort_values(by=['src_sub_area','datetime'], inplace=True)

scotty_group = scotty_group.groupby('src_sub_area').\
    apply(lambda x : x.set_index('datetime').resample('H').sum().fillna(0)).\
    reset_index()

sxk97 = scotty_group[scotty_group.src_sub_area == 'sxk97']
sxk9e = scotty_group[scotty_group.src_sub_area == 'sxk9e']
sxk9s = scotty_group[scotty_group.src_sub_area == 'sxk9s']

In [ ]:
sxk97 = sxk97.drop(columns=['src_sub_area'])
sxk97.set_index('datetime', inplace=True)

In [ ]:
df = sxk97
# df.shape
df.head()

## From MachineLearningMastery

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

scaler.fit(train_data)
scaled_train_data = scaler.transform(train_data)
scaled_test_data = scaler.transform(test_data)

In [ ]:
scaled_train_data.shape

In [ ]:
# univariate multi-step vector-output stacked lstm example
from numpy import array
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense

# split a univariate sequence into samples
def split_sequence(sequence, n_steps_in, n_steps_out):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out
        # check if we are beyond the sequence
        if out_end_ix > len(sequence):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix:out_end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

# define input sequence
# raw_seq = [10, 20, 30, 40, 50, 60, 70, 80, 90]
raw_seq = train_data['demand'].tolist()
# choose a number of time steps
n_steps_in, n_steps_out = 24*7, 2
# n_steps_in, n_steps_out = n_test*3, n_test
# split into samples
X, y = split_sequence(raw_seq, n_steps_in, n_steps_out)
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))
# X = raw.reshape(raw.shape[1], raw.shape[0], n_features)
# y = test_data.values
# y = y.reshape(y.shape[1], y.shape[0])

In [ ]:
# define model
model = Sequential()
model.add(LSTM(100, activation='relu', return_sequences=True, input_shape=(n_steps_in, n_features)))
model.add(LSTM(100, activation='relu'))
model.add(Dense(n_steps_out))
model.compile(optimizer='adam', loss='mae')
# fit model
model.fit(X, y, epochs=2, verbose=1)

In [ ]:
# model.summary()
print(X.shape)
print(y.shape)

In [ ]:
losse = model.history.history['loss']
plt.plot(range(len(losse)), losse)
plt.show()
# losse

In [ ]:
# demonstrate prediction
x_input = array([70, 80, 90])
x_input = x_input.reshape((1, n_steps_in, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)

In [ ]:
[[120.097084 138.88902 ]]
[[116.9725  131.78836]]

In [ ]:
# split a univariate sequence into samples
def split_sequence(sequence, n_steps_in, n_steps_out):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out
        # check if we are beyond the sequence
        if out_end_ix > len(sequence):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix:out_end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

In [ ]:
n_test = 24*7
train_data = df[:len(df)-n_test]
test_data = df[len(df)-n_test:]

In [ ]:
# define input sequence
raw_seq = [10, 20, 30, 40, 50, 60, 70, 80, 90]
# choose a number of time steps
n_steps_in, n_steps_out = 3, 2
# split into samples
X, y = split_sequence(raw_seq, n_steps_in, n_steps_out)
# summarize the data
for i in range(len(X)):
	print(X[i], y[i])

## From TowardsDataScience

In [ ]:
train_data = df[:len(df)-252]
test_data = df[len(df)-252:]

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [ ]:
scaler.fit(train_data)
scaled_train_data = scaler.transform(train_data)
scaled_test_data = scaler.transform(test_data)

In [ ]:
from keras.preprocessing.sequence import TimeseriesGenerator

n_input = 1000
n_features= 1
generator = TimeseriesGenerator(scaled_train_data, scaled_train_data, length=n_input, batch_size=252)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

lstm_model = Sequential()
lstm_model.add(LSTM(10, activation='tanh', input_shape=(n_input, n_features)))
lstm_model.add(Dropout(0.2))
lstm_model.add(Dense(1))
lstm_model.compile(optimizer='adam', loss='mse')

lstm_model.summary()

In [ ]:
lstm_model.fit_generator(generator,epochs=50)

In [ ]:
losses_lstm = lstm_model.history.history['loss']
plt.figure(figsize=(12,4))
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.xticks(np.arange(0,21,1))
plt.plot(range(len(losses_lstm)),losses_lstm);

In [ ]:
lstm_predictions_scaled = list()

batch = scaled_train_data[-n_input:]
current_batch = batch.reshape((1, n_input, n_features))

for i in range(len(test_data)):   
    lstm_pred = lstm_model.predict(current_batch)[0]
    lstm_predictions_scaled.append(lstm_pred) 
    current_batch = np.append(current_batch[:,1:,:],[[lstm_pred]],axis=1)

In [ ]:
aa = lstm_model.predict(scaled_train_data.reshape(1, 1260, n_features))
# aa = scaler.inverse_transform(aa)
# plt.plot(train_data.index, aa)

In [ ]:
lstm_predictions_scaled

In [ ]:
lstm_predictions = scaler.inverse_transform(lstm_predictions_scaled)

In [ ]:
lstm_predictions

In [ ]:
test_data['LSTM pred'] = lstm_predictions
test_data.head()

In [ ]:
# plt.plot(test_data.index, test_data['demand'])
plt.plot(test_data.index, test_data['LSTM pred'])
plt.show()

### Based on TF web

In [ ]:
import tensorflow as tf

In [ ]:
def univariate_data(dataset, start_index, end_index, history_size, target_size):
  data = []
  labels = []

  start_index = start_index + history_size
  if end_index is None:
    end_index = len(dataset) - target_size

  for i in range(start_index, end_index):
    indices = range(i-history_size, i)
    # Reshape data from (history_size,) to (history_size, 1)
    data.append(np.reshape(dataset[indices], (history_size, 1)))
    labels.append(dataset[i+target_size])
  return np.array(data), np.array(labels)


In [ ]:
uni_data = sxk97.values
TRAIN_SPLIT = 1000

In [ ]:
uni_train_mean = uni_data[:TRAIN_SPLIT].mean()
uni_train_std = uni_data[:TRAIN_SPLIT].std()

In [ ]:
uni_data = (uni_data-uni_train_mean)/uni_train_std

In [ ]:
univariate_past_history = 20
univariate_future_target = 0

x_train_uni, y_train_uni = univariate_data(uni_data, 0, TRAIN_SPLIT,
                                           univariate_past_history,
                                           univariate_future_target)
x_val_uni, y_val_uni = univariate_data(uni_data, TRAIN_SPLIT, None,
                                       univariate_past_history,
                                       univariate_future_target)

In [ ]:
BATCH_SIZE = 256
BUFFER_SIZE = 10000

train_univariate = tf.data.Dataset.from_tensor_slices((x_train_uni, y_train_uni))
train_univariate = train_univariate.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()

val_univariate = tf.data.Dataset.from_tensor_slices((x_val_uni, y_val_uni))
val_univariate = val_univariate.batch(BATCH_SIZE).repeat()

In [ ]:
simple_lstm_model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(8, input_shape=x_train_uni.shape[-2:]),
    tf.keras.layers.Dense(1)
])

simple_lstm_model.compile(optimizer='adam', loss='mae')


In [ ]:
EVALUATION_INTERVAL = 200
EPOCHS = 100

simple_lstm_model.fit(train_univariate, epochs=EPOCHS,
                      steps_per_epoch=EVALUATION_INTERVAL,
                      validation_data=val_univariate, validation_steps=50)


In [ ]:
for x, y in val_univariate.take(3):
  plot = show_plot([x[0].numpy(), y[0].numpy(),
                    simple_lstm_model.predict(x)[0]], 0, 'Simple LSTM model')
  plot.show()


In [ ]:
losses_TF = simple_lstm_model.history.history['loss']
plt.plot(range(len(losses_TF)), losses_TF)

# losses_lstm = lstm_model.history.history['loss']
# plt.figure(figsize=(12,4))
# plt.xlabel("Epochs")
# plt.ylabel("Loss")
# plt.xticks(np.arange(0,21,1))
# plt.plot(range(len(losses_lstm)),losses_lstm);

In [ ]:
# plot = show_plot([x[0].numpy(), y[0].numpy(),
#                     simple_lstm_model.predict(x)[0]], 0, 'Simple LSTM model')
# plot.show()

pred = simple_lstm_model.predict(x_val_uni)
plt.plot()